In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from scipy.optimize import curve_fit

from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [2]:
LS133 = "./ntupleStubsBmtfGmt_LS133.root"
LS263 = "./ntupleStubsBmtfGmt_LS263.root"
LS264 = "./ntupleStubsBmtfGmt_LS264.root"
LS306 = "./ntupleStubsBmtfGmt_LS306.root"

#Open file
stubs_awk_LS133 = OpenRootFile(LS133, stub_features)
stubs_awk_LS263 = OpenRootFile(LS263, stub_features)
stubs_awk_LS264 = OpenRootFile(LS264, stub_features)
stubs_awk_LS306 = OpenRootFile(LS306, stub_features)

gmt_awk_LS133 = OpenRootFile(LS133, gmt_features)
gmt_awk_LS263 = OpenRootFile(LS263, gmt_features)
gmt_awk_LS264 = OpenRootFile(LS264, gmt_features)
gmt_awk_LS306 = OpenRootFile(LS306, gmt_features)

bmtf_awk_LS133 = OpenRootFile(LS133, bmtf_features)
bmtf_awk_LS263 = OpenRootFile(LS263, bmtf_features)
bmtf_awk_LS264 = OpenRootFile(LS264, bmtf_features)
bmtf_awk_LS306 = OpenRootFile(LS306, bmtf_features)

In [3]:
#Caricamenot BX delle stubs
BX_LS133 = stubs_awk_LS133.bx
BX_LS263 = stubs_awk_LS263.bx
BX_LS264 = stubs_awk_LS264.bx
BX_LS306 = stubs_awk_LS306.bx

#Caricamento muoni di GMT
nGmtMuons_LS133 = gmt_awk_LS133.nGmtMuons
nGmtMuons_LS263 = gmt_awk_LS263.nGmtMuons
nGmtMuons_LS264 = gmt_awk_LS264.nGmtMuons
nGmtMuons_LS306 = gmt_awk_LS306.nGmtMuons

nBmtfMuons_LS133 = bmtf_awk_LS133.nBmtfMuons
nBmtfMuons_LS263 = bmtf_awk_LS263.nBmtfMuons
nBmtfMuons_LS264 = bmtf_awk_LS264.nBmtfMuons
nBmtfMuons_LS306 = bmtf_awk_LS306.nBmtfMuons

In [38]:
#GMT

weightsNGmt = ak.where(nBmtfMuons_LS133 <= 1, nBmtfMuons_LS133, 1)

bx_gmt_counts_LS133 = np.bincount(BX_LS133, weights=weightsNGmt)
bx_gmt_counts_LS263 = np.bincount(BX_LS263, weights=nGmtMuons_LS263)
bx_gmt_counts_LS264 = np.bincount(BX_LS264, weights=nGmtMuons_LS264)
bx_gmt_counts_LS306 = np.bincount(BX_LS306, weights=nGmtMuons_LS306)

#bx_gmt_counts = (bx_gmt_counts_LS133 + bx_gmt_counts_LS263 + bx_gmt_counts_LS264 + bx_gmt_counts_LS306)/4


#BMTF
weightsN = ak.where(nBmtfMuons_LS133 <= 1, nBmtfMuons_LS133, 1)


bx_bmtf_counts_LS133N = np.bincount(BX_LS133, weights=weightsN)
bx_bmtf_counts_LS133 = np.bincount(BX_LS133, weights=nBmtfMuons_LS133)

bx_bmtf_counts_LS263 = np.bincount(BX_LS263, weights=nBmtfMuons_LS263)
bx_bmtf_counts_LS264 = np.bincount(BX_LS264, weights=nBmtfMuons_LS264)
bx_bmtf_counts_LS306 = np.bincount(BX_LS306, weights=nBmtfMuons_LS306)

#bx_bmtf_counts = (bx_bmtf_counts_LS133 + bx_bmtf_counts_LS263 + bx_bmtf_counts_LS264 + bx_bmtf_counts_LS306)/4

print(bx_bmtf_counts_LS133N[100])
print(bx_bmtf_counts_LS133[100])
print(bx_gmt_counts_LS133[100])


193.0
204.0
195.0


In [5]:
def BX_OPT(BX, nMuons):

    BX_Matrix = [[] for _ in range(ONE_ORBIT_IN_BX)]
    
    for j in range(len(BX)):

      if nMuons[j] > 0:
        BX_Matrix[BX[j]].append([j, nMuons[j]])
    
    BX_Matrix = ak.Array(BX_Matrix)

    return BX_Matrix

In [6]:
BX_BMTF_LS133 = BX_OPT(BX_LS133, nBmtfMuons_LS133)
BX_GMT_LS133 = BX_OPT(BX_LS133, nGmtMuons_LS133)

In [7]:
'''for i in range(100):
  print(i, " ", BX_BMTF_LS133[i])'''

'for i in range(100):\n  print(i, " ", BX_BMTF_LS133[i])'

In [8]:
GmtPhi_LS133 = gmt_awk_LS133.gmtPhi
GmtEta_LS133 = gmt_awk_LS133.gmtEta

BmtfEta_LS133 = bmtf_awk_LS133.bmtfEta
BmtfPhi_LS133 = bmtf_awk_LS133.bmtfPhi

GmtEta_LS133 = ak.where(ak.num(GmtEta_LS133) == 0, ak.Array([[0]]), GmtEta_LS133)
GmtPhi_LS133 = ak.where(ak.num(GmtPhi_LS133) == 0, ak.Array([[0]]), GmtPhi_LS133)
BmtfEta_LS133 = ak.where(ak.num(BmtfEta_LS133) == 0, ak.Array([[0]]), BmtfEta_LS133)
BmtfPhi_LS133 = ak.where(ak.num(BmtfPhi_LS133) == 0, ak.Array([[0]]), BmtfPhi_LS133)


In [9]:
print(BX_BMTF_LS133[100, -3])

[2854073, 2]


In [10]:
print(len(BX_GMT_LS133[100]))

193


In [17]:
def DeltaR(BmtfPhi, GmtPhi, BmtfEta, GmtEta):
  return np.sqrt((BmtfPhi - GmtPhi)**2 + (BmtfEta - GmtEta)**2)

def DeltaRDistribution(BX_BMTF, BX_GMT, BmtfPhi, GmtPhi, BmtfEta, GmtEta, use_Bmtf = True):

  DeltaDist = [[] for _ in range(ONE_ORBIT_IN_BX)]
  IndexMin = [[] for _ in range(ONE_ORBIT_IN_BX)]

  if use_Bmtf: 
    for i in range(ONE_ORBIT_IN_BX):

      NestedArrayDeltaR = []
      NestedArrayIndex = []

      BmtfPhiF, BmtfEtaF = ak.flatten(BmtfPhi[BX_BMTF[i, :, 0]]), ak.flatten(BmtfEta[BX_BMTF[i, :, 0]])
      GmtPhiF, GmtEtaF = ak.flatten(GmtPhi[BX_GMT[i, :, 0]]), ak.flatten(GmtEta[BX_GMT[i, :, 0]])

      #Crea un array con il numero di eventi all'interno di ogni sottoarry, serve per ricostruitre la struttura iniziale
      Bmtf_counts = ak.num(BmtfPhi[BX_BMTF[i, :, 0]])
      Gmt_counts = ak.num(GmtPhi[BX_GMT[i, :, 0]])

      for j in range(len(BmtfPhiF)):

        DeltaRTemp = DeltaR(BmtfPhiF[j], GmtPhiF[:], BmtfEtaF[j], GmtEtaF[:])

        minDeltaR = np.min(DeltaRTemp)
        #indexTemp = np.argmin(DeltaRTemp)

        NestedArrayDeltaR.append(minDeltaR)
        #NestedArrayIndex.append(indexTemp)

      DeltaDist[i].extend(NestedArrayDeltaR)
      #IndexMin[i].extend(NestedArrayIndex)

  return ak.Array(DeltaDist)#, ak.Array(IndexMin)

In [18]:
RDistribution = DeltaRDistribution(BX_BMTF_LS133, BX_GMT_LS133, BmtfPhi_LS133, GmtPhi_LS133, BmtfEta_LS133, GmtEta_LS133, use_Bmtf = True)

In [34]:
RDistributionF = ak.flatten(RDistribution)
[x for x in range(len(RDistributionF)) if RDistributionF[x] > 0.3]

[22283, 24626, 29636, 83509, 184019, 248916, 492124]

In [56]:
i = 100
BmtfPhiF, BmtfEtaF = ak.flatten(BmtfPhi_LS133[BX_BMTF_LS133[i, :, 0]]), ak.flatten(BmtfEta_LS133[BX_BMTF_LS133[i, :, 0]])
GmtPhiF, GmtEtaF = ak.flatten(GmtPhi_LS133[BX_GMT_LS133[i, :, 0]]), ak.flatten(GmtEta_LS133[BX_GMT_LS133[i, :, 0]])

for j in range(100):
  DeltaRTemp = DeltaR(BmtfPhiF[j], GmtPhiF[:], BmtfEtaF[j], GmtEtaF[:])
  print(np.argmin(DeltaRTemp), " ", np.min(DeltaRTemp), " ", DeltaRTemp[np.argmin(DeltaRTemp)])

  


0   0.0   0.0
1   0.0   0.0
2   0.0   0.0
3   0.0   0.0
4   0.0   0.0
5   0.0   0.0
6   0.0   0.0
7   0.0   0.0
8   0.0   0.0
9   0.0   0.0
10   0.0   0.0
11   0.010875016450881958   0.010875016450881958
12   0.0   0.0
13   0.0   0.0
14   0.0   0.0
15   0.0   0.0
16   0.0   0.0
17   0.0   0.0
17   0.0   0.0
18   0.0   0.0
19   0.0   0.0
20   0.0   0.0
21   0.0   0.0
22   0.0   0.0
23   0.0   0.0
24   0.0   0.0
25   0.0   0.0
26   0.0   0.0
26   0.0   0.0
27   0.0   0.0
28   0.0   0.0
29   0.010875001549720764   0.010875001549720764
30   0.0   0.0
31   0.0   0.0
32   0.0   0.0
33   0.0   0.0
34   0.0   0.0
35   0.09787499904632568   0.09787499904632568
35   0.0   0.0
36   0.0   0.0
37   0.0   0.0
38   0.0   0.0
39   0.0   0.0
40   0.0   0.0
41   0.010875046253204346   0.010875046253204346
42   0.0   0.0
43   0.0   0.0
44   0.0   0.0
45   0.0   0.0
46   0.0   0.0
46   0.0   0.0
47   0.0   0.0
48   0.0   0.0
49   0.0   0.0
50   0.0   0.0
51   0.0   0.0
52   0.0   0.0
53   0.01087498664855

In [29]:
i = 100
Bmtf_counts = ak.num(BmtfPhi_LS133[BX_BMTF_LS133[i, :, 0]])

print(len(Bmtf_counts))

193


In [13]:
i = 100
j = -3
'''print(BmtfPhi_LS133[BX_BMTF_LS133[i, j]])
print(BX_BMTF_LS133[i, j])
print(len(GmtPhi_LS133[BX_GMT_LS133[i, :, 0]]))'''



BmtfPhi_LS133M = ak.flatten(BmtfPhi_LS133[BX_BMTF_LS133[i, :, 0]])
print(BmtfPhi_LS133M)
'''for j in range(len(BX_GMT_LS133[i])):
  print(GmtPhi_LS133[BX_GMT_LS133[i, j, 0]])'''
'''print(len(ak.flatten(BmtfPhi_LS133[BX_BMTF_LS133[i, :, 0]])))

print(len(ak.flatten(GmtPhi_LS133[BX_GMT_LS133[i, :, 0]])))'''

[0.665, 2.57, -1.6, -1.27, 1.91, 1.13, ..., -1.46, 1.65, 2.45, -2.3, 3.09]


'print(len(ak.flatten(BmtfPhi_LS133[BX_BMTF_LS133[i, :, 0]])))\n\nprint(len(ak.flatten(GmtPhi_LS133[BX_GMT_LS133[i, :, 0]])))'

In [25]:
import awkward as ak
import numpy as np

# Array originale annidato
original = ak.Array([[1, 2, 3], [], [4, 5], [6]])

print("Array originale:")
print(original)

# Salva la struttura originale
original_counts = ak.num(original)

# Appiattisci l'array
flattened = ak.flatten(original)

print("\nArray appiattito:")
print(flattened)

# Ricostruisci l'array usando ak.unflatten()
reconstructed = ak.unflatten(flattened, original_counts)

print("\nArray ricostruito con ak.unflatten():")
print(reconstructed)

# Alternativa usando ak.partition()
partitioned = ak.partition(flattened, original_counts)

print("\nArray ricostruito con ak.partition():")
print(partitioned)

# Verifica che le ricostruzioni siano uguali all'originale
print("\nLa ricostruzione con unflatten è uguale all'originale?")
print(ak.all(reconstructed == original))

print("\nLa ricostruzione con partition è uguale all'originale?")
print(ak.all(partitioned == original))

Il massimo si trova in posizione: ([3 4 3 4], [0 1 0 1])
